In [ ]:
def detect_slow_windows(logs: list[str], slow_ms: int, min_requests: int) -> list[tuple[str, int]]:
    grouping = {}
    result = []
    for log in logs:
        log = log.strip()
        if not log or log.startswith("#"):
            continue
        
        ts, t_id, resp_t = log.split(",")

        ts = int(ts)
        resp_t = int(resp_t)
        
        window_t = (ts // 60) * 60
        
        key = (t_id, window_t) #1700000100, tA
        
        if key not in grouping:
            grouping[key] = (0,0)
        
        (count,slow) = grouping[key] #1700000100, tA--> 0,0
        
        count += 1
        
        if resp_t >= slow_ms:
            slow += 1
            
        grouping[key] = (count,slow) #{(tA, 1700000100): (4,2),(tB, 1700000100): (4,0) }
        
        
        #[((tA, 1700000100), (4,2)),((tB, 1700000100), (4,0)) ]
    for (tenant_id, window_t),(count,slow) in grouping.items():
        if count >= min_requests:
            if slow / count == 0.5:
                result.append((tenant_id, window_t))
                    
    return result
        
print(detect_slow_windows([
    "1700000101,tenantA,100",
    "1700000102,tenantA,900",
    "1700000103,tenantA,850",
    "1700000104,tenantA,200",

    "1700000120,tenantB,700",
    "1700000121,tenantB,100",
    "1700000122,tenantB,100",
    "1700000123,tenantB,100"
], 800,4))

In [ ]:
# slow_ms = 800
# min_requests = 4

# Expected output:
# [("tenantA", 1700000100)]

# Because:
# * `tenantA` in window `1700000100` has 4 requests
# * 2 of them are slow (`900`, `850`)
#2 / 4 = 0.5, so it qualifies

In [ ]:
def detect_return_spikes(logs: list[str], return_rate_threshold: float, min_actions: int) -> list[tuple[str, int]]:
    
    
    
# logs = [
#     "1700000101,custA,BUY",
#     "1700000102,custA,RETURN",
#     "1700000103,custA,RETURN",
#     "1700000104,custA,BUY",

#     "1700000120,custB,RETURN",
#     "1700000121,custB,BUY",
#     "1700000122,custB,BUY",
#     "1700000123,custB,BUY",

#     "1700000165,custA,RETURN",
#     "1700000166,custA,RETURN",
#     "1700000167,custA,BUY"
# ]
# return_rate_threshold = 0.5
# min_actions = 4

# Then expected output is:
# [("custA", 1700000100)]
    
    
# Return all (customer_id, window_start) where:
# the window is a 60-second bucket
# that customer has at least min_actions total actions in that window
# the return rate in that window is at least return_rate_threshold